In [3]:
import json
import os
from pathlib import Path

# STEP 0: Open config.json
with open("config.json", "r") as f:
    config = json.load(f)

DATA_DIR = config["data_dir"]
DOWNLOAD_MISSING_DATASET = config["download_missing_dataset"]
GENRES = config["genres"]
INSTALL_MISSING_LIBRARIES = config["install_missing_libraries"]
LIBRARIES = config["libraries"]

script_path = Path("src")
script_download_missing_library = os.path.join(script_path, "download_missing_library.py")

# STEP 1: Check mandatory libraries to run the notebooks
print(f"Find install_missing_libraries = {INSTALL_MISSING_LIBRARIES} in config.json")
if INSTALL_MISSING_LIBRARIES:
    print(f"Missing libraries will be installed automatically")
else:
    print(f"Missing libraries will not be installed automatically")

for pkg in LIBRARIES:
    try:
        __import__(pkg)
    except ImportError:
        if INSTALL_MISSING_LIBRARIES:
            print(f"Installing {pkg}")
            !python "{script_download_missing_library}" {pkg}
        else:
            raise ImportError(f"Mandatory library not found: {pkg}."
                              f"Run the following command to install it: pip install {pkg} or set install_missing_libraries = true in config.json")

# STEP 2: Check the current working directory and validate it with an assertion
assert os.path.basename(os.getcwd()) == "GTZAN_Genre_Recognizer", "The script should be starded from root-project folder with GTZAN_Genre_Recognizer name"

# STEP 3: Run all the notebooks in sequence
%run notebooks/01_data_setup.ipynb ##TODO: Test download_data.py and invoke it when dataset is not valid (priority very-low)
%run notebooks/02_features.ipynb
#%run 03_training.ipynb
#%run 04_evaluation.ipynb

Find install_missing_libraries = True in config.json
Missing libraries will be installed automatically
System detect: Windows
Actual directory:  c:\Users\rpork\Desktop\Documenti\Studio\MAGISTRALE\Machine Learning\GTZAN_Genre_Recognizer
Check if data exists at the following path:  data\genres
All subfolders found
Number of tracks founds:
blues     : 100
classical : 100
country   : 100
disco     : 100
hiphop    : 100
jazz      : 100
metal     : 100
pop       : 100
reggae    : 100
rock      : 100
Total tracks: 1000
